In [70]:
import torch
import torch.nn as nn
from utils.auxfunctions import moving_window
import numpy as np
import matplotlib.pyplot as plt

In [56]:
hidden_dim = 6
n_signals = 1
N = 64
tagset_size = 1

In [50]:
class LSTM_freq(nn.Module):
    """Some Information about LSTM_freq"""
    def __init__(self, n_signals, hidden_dim, tagset_size):
        super(LSTM_freq, self).__init__()
        self.lstm = nn.LSTM(n_signals, hidden_dim)
        self.fc = nn.Linear(hidden_dim, tagset_size)
        self.activation = nn.Sigmoid()
        

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        print(lstm_out.shape)
        fc_layer = self.fc(lstm_out[-1])
        print(fc_layer.shape)
        prob_tag = self.activation(fc_layer)
        print(prob_tag.shape)

        return prob_tag


In [51]:
model = LSTM_freq(n_signals, hidden_dim, tagset_size)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)


In [54]:
mock_dataset = torch.rand(64, 5, 1)
with torch.no_grad():
    tag_scores = model(mock_dataset)
    print(tag_scores)

torch.Size([64, 5, 6])
torch.Size([5, 1])
torch.Size([5, 1])
tensor([[0.4710],
        [0.4729],
        [0.4726],
        [0.4714],
        [0.4718]])


In [69]:
fs = 3840
t = np.linspace(0,1/60,64)
signal = np.sin(2*np.pi *fs*t)
print(signal)

[ 0.00000000e+00  9.95678466e-02  1.98146143e-01  2.94755174e-01
  3.88434796e-01  4.78253979e-01  5.63320058e-01  6.42787610e-01
  7.15866849e-01  7.81831482e-01  8.40025923e-01  8.89871809e-01
  9.30873749e-01  9.62624247e-01  9.84807753e-01  9.97203797e-01
  9.99689182e-01  9.92239207e-01  9.74927912e-01  9.47927346e-01
  9.11505852e-01  8.66025404e-01  8.11938006e-01  7.49781203e-01
  6.80172738e-01  6.03804410e-01  5.21435203e-01  4.33883739e-01
  3.42020143e-01  2.46757398e-01  1.49042266e-01  4.98458857e-02
 -4.98458857e-02 -1.49042266e-01 -2.46757398e-01 -3.42020143e-01
 -4.33883739e-01 -5.21435203e-01 -6.03804410e-01 -6.80172738e-01
 -7.49781203e-01 -8.11938006e-01 -8.66025404e-01 -9.11505852e-01
 -9.47927346e-01 -9.74927912e-01 -9.92239207e-01 -9.99689182e-01
 -9.97203797e-01 -9.84807753e-01 -9.62624247e-01 -9.30873749e-01
 -8.89871809e-01 -8.40025923e-01 -7.81831482e-01 -7.15866849e-01
 -6.42787610e-01 -5.63320058e-01 -4.78253979e-01 -3.88434796e-01
 -2.94755174e-01 -1.98146

In [61]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:]} \n")

Model structure: LSTM_freq(
  (lstm): LSTM(1, 6)
  (fc): Linear(in_features=6, out_features=1, bias=True)
  (activation): Sigmoid()
)


Layer: lstm.weight_ih_l0 | Size: torch.Size([24, 1]) | Values : tensor([[-0.2381],
        [ 0.0176],
        [-0.1039],
        [-0.2774],
        [ 0.2367],
        [-0.3029],
        [ 0.3260],
        [-0.0017],
        [-0.0507],
        [ 0.3011],
        [ 0.0988],
        [ 0.3949],
        [-0.0184],
        [-0.3725],
        [ 0.0911],
        [ 0.3168],
        [-0.2602],
        [-0.1913],
        [ 0.0217],
        [ 0.3465],
        [-0.3551],
        [ 0.0652],
        [-0.1206],
        [ 0.2366]], grad_fn=<SliceBackward0>) 

Layer: lstm.weight_hh_l0 | Size: torch.Size([24, 6]) | Values : tensor([[ 0.3410,  0.3745,  0.0846, -0.0683, -0.1805,  0.3230],
        [-0.0581, -0.0646,  0.1498, -0.1316, -0.2244, -0.4081],
        [ 0.1802, -0.0735, -0.3571,  0.1071,  0.3528,  0.3861],
        [ 0.2982, -0.3160, -0.2787, -0.3930, -0.1771, -0.36